# Data Preprocessing

In [ ]:
# Pandas 및 Numpy 패키지 불러오기
import pandas as pd 
import numpy as np
# 청소년 건강행태조사 2016년, 2017년, 2018년도 데이터 불러오기
df16_og = pd.read_sas('/content/drive/My Drive/고급데이터마이닝/청건행 데이터셋/kyrbs2016.sas7bdat')
df17_og = pd.read_sas('/content/drive/My Drive/고급데이터마이닝/청건행 데이터셋/kyrbs2017.sas7bdat')
df18_og = pd.read_sas('/content/drive/My Drive/ 고급데이터마이닝/청건행 데이터셋 /kyrbs2018.sas7bdat')

# 분석용 데이터셋 저장하기
df16 = df16_og.copy()
df17 = df17_og.copy()
df18 = df18_og.copy()

## 청소년 건강행태조사 데이터 전처리

In [ ]:
# 청소년 건강행태조사 파일 하나로 합치기
cgh = pd.concat([df16, df17, df18])

# 선행 연구(논문) 기반 유의미한 변수 추출
df_att = cgh[["sex","age","D_1_1","Total_slp_wd","BP1","BO1","BE5_1","BE8_1","BE8_2","BP5","HE_ht","HE_wt"]]

#히스토그램 산출 ( 탐색적 분석 )
import matplotlib.pyplot as plt
import seaborn as sns
for df in ["sex","age","D_1_1","Total_slp_wd","BP1","BO1","BE5_1","BE8_1","BE8_2","BP5","HE_ht","HE_wt"]:
    plt.hist(df_att[df])
    plt.ylabel('frequency')
    plt.title('Histogram of {}'.format(df))
    plt.show()

# 분석에 사용할 변수만 추출
cgh = cgh.loc[:,['HT','WT','SEX','AGE','M_SLP_HR','M_SLP_MM','M_WK_HR','M_WK_MM','M_SLP_HR_K','M_SLP_MM_K','M_WK_HR_K','M_WK_MM_K','PR_HT','M_STR','PR_BI','PA_MSC','PA_SWD_S','PA_SWD_N','PA_SWK_S','PA_SWK_N','M_SAD']]

# 청소년 건강행태조사 분석 대상 데이터셋 Instance 개수 및 변수 개수 확인
cgh.shape

In [ ]:
# 앉은시간 변수 생성
cgh['SitTime'] = ((cgh['PA_SWD_S'] + cgh['PA_SWD_N'])*5 + (cgh['PA_SWK_S'] + cgh['PA_SWK_N'])*2)/7
cgh['SitTime'].describe()


# 앉은시간 boxplot
cgh.boxplot(column = 'SitTime')

# 인덱스 초기화
cgh = cgh.reset_index()
cgh = cgh.iloc[:,1:]
cgh.shape

In [ ]:
# 주중 (월~금) 수면시간 전처리
for i in range(len(cgh["M_SLP_HR"])):
    if cgh["M_SLP_HR"][i]<=12:
        cgh["M_SLP_HR"][i] += 24
    else:
        pass

    if cgh["M_WK_HR"][i]<=12:
        cgh["M_WK_HR"][i] +=24
    else:
        pass

for i in range(len(cgh["M_SLP_HR"])):
    cgh["SLP_WK"] = (cgh["M_WK_HR"]*60 + cgh["M_WK_MM"])-(cgh["M_SLP_HR"]*60 + cgh["M_SLP_MM"])

# 생선된 변수 기초통계량 확인 (Instance 개수, 평균값, 4분위수 등)
cgh["SLP_WK"].describe()

In [ ]:
# 평소 스테레스 인지 정도에서 값이 5인 행들 삭제하기 (국민건강영양조사 데이터와 합치기 위한 작업)
idx_nm = cgh[cgh["M_STR"] == 5].index
cgh = cgh.drop(idx_nm)
cgh.shape

In [ ]:
# 2주 이상 연속 우울감 여부 값 바꾸기
cgh = cgh.replace({"M_SAD":1},{"M_SAD":3})
cgh = cgh.replace({"M_SAD":2},{"M_SAD":1})
cgh = cgh.replace({"M_SAD":3},{"M_SAD":2})
cgh["M_SAD"].value_counts() # 1:예   2:아니오

In [ ]:
# BMI [BMI] (명목) 변수 생성
cgh['BMI'] = (cgh['WT'] / (cgh['HT'] / 100)**2)
cgh.loc[cgh['BMI'] < 25, 'BMI'] = 0
cgh.loc[cgh['BMI'] >= 25, 'BMI'] = 1
cgh['BMI'] = cgh['BMI'].astype('category')
print(cgh['BMI'].value_counts())
print("==========================")
print(cgh['BMI'])

In [ ]:
# 청건행 변수명을 국건영과 동일하게 바꾸기
df = cgh.rename(columns = {'SEX':'sex','AGE':'age','PR_HT':'D_1_1','M_STR':'BP1','PR_BI':'BO1','PA_MSC':'BE5_1','M_SAD':'BP5', 'SLP_WK':'Total_slp_wd'})

# 필요한 Column만 불러오기
df_cgh = df.loc[:,['sex', 'age', 'D_1_1', 'BP1', 'BO1', 'BE5_1', 'BP5', 'BMI', 'Total_slp_wd', 'SitTime']]
df_cgh.shape

## 국민 건강 영양조사 데이터 전처리

In [ ]:
df_16 = pd.read_sas('/content/drive/My Drive/고급데이터마이닝/국건영 원본 데이터셋/hn16_all.sas7bdat')
df_16

df_17 = pd.read_sas('/content/drive/My Drive/고급데이터마이닝/국건영 원본 데이터셋/hn17_all.sas7bdat')
df_17

df_18 = pd.read_sas('/content/drive/My Drive/고급데이터마이닝/국건영 원본 데이터셋/hn18_all.sas7bdat')
df_18

# 국민건강영양조사 2016년 ~ 2018년 데이터 하나로 합치기
# 추후에는 국민건강영양조사 2018년도 데이터만 사용함
df_kky = pd.concat([df_16,df_17,df_18])

# 선행 연구(논문) 기반 유의미한 변수 추출
df_att = df_kky[["sex","age","D_1_1","Total_slp_wd","BP1","BO1","BE5_1","BE8_1","BE8_2","BP5","HE_ht","HE_wt"]]

#히스토그램 산출 ( 탐색적 분석 )
import matplotlib.pyplot as plt
import seaborn as sns
for df in ["sex","age","D_1_1","Total_slp_wd","BP1","BO1","BE5_1","BE8_1","BE8_2","BP5","HE_ht","HE_wt"]:
    plt.hist(df_att[df])
    plt.ylabel('frequency')
    plt.title('Histogram of {}'.format(df))
    plt.show()


## 청건행 데이터 및 국건영 데이터 합친 후 최종 전처리

In [ ]:
# 분석에 사용한 변수만 추출하여 새로운 데이터셋 구성
df_att = df_kky[["sex","age","D_1_1","Total_slp_wk","BP1","BO1","BE5_1","BE8_1","BE8_2","BP5","HE_ht","HE_wt"]]

# 주중(Weekday)라는 단어와 혼동이 있어서 Total_slp_wk를 Total_slp_wd로 변경
df_att.rename(columns = {'Total_slp_wk': 'Total_slp_wd'), inplace = True)  

# 인덱스 초기화
df_att = df_att.reset_index()
df_att = df_att.iloc[:,1:]
df_att.shape
                                                                       
# 변수별 Instance, 결측치, 데이터 타입 확인
df_att.info()
# BMI [BMI] (명목) 변수 생성
df_att['BMI'] = (df_att['HE_wt'] / (df_att['HE_ht'] / 100)**2)
df_att.loc[df_att['BMI'] < 25, 'BMI'] = 0
df_att.loc[df_att['BMI'] >= 25, 'BMI'] = 1
print(df_att['BMI'].value_counts())
print("==========================")
print(df_att['BMI'])

# 청소년 (만12세~18세) 데이터 생성
df_att = df_att.loc[(df_att['age']>=12) & (df_att['age']<=18), :]  
df_att.info()

df_att.loc[(df_att['BE8_1'] == 88) | (df_att['BE8_1'] == 99) , 'BE8_1'] = np.nan 
df_att.loc[(df_att['BE8_2'] == 88) | (df_att['BE8_2'] == 99) , 'BE8_2'] = np.nan 

# 하루 평균 앉아서 보내는 시간(분 단위) 변수 생성
df_att["SitTime"] = df_att["BE8_1"]*60 + df_att['BE8_2']
df_att["SitTime"].describe()

# 전처리된 청소년건강행태조사 데이터 및 국민건강영양조사 데이터 합치기
df_final = pd.concat([df_att,df_cgh])
df_final.shape

# 최종 분석 대상 데이터 변수, 변수별 Instance 및 결측치 개수, 데이터 타입 확인 
df_final.info()

df_final['BMI'] = df_final['BMI'].astype('category')

df_final = df_final.loc[:,['sex','age','D_1_1','BP1','BO1','BE5_1','BP5','BMI','Total_slp_wk','SitTime']]

# 1주일간 근력운동 일수 (등간)
print(df_final['BE5_1'].value_counts()) # 8 비해당 929명, 9 모름, 무응답 : 267명 -> NA
df_final.loc[(df_final['BE5_1'] == 8) |(df_final['BE5_1'] == 9) , 'BE5_1'] = np.nan  #na 값 처리
print(df_final['BE5_1'].value_counts())

# 주관적 건강상태 [D_1_1]] (서열)
from pandas.api.types import CategoricalDtype
print(df_final['D_1_1'].value_counts())
print("==========================")
df_final.loc[df_final['D_1_1'] == 9, 'D_1_1'] = np.nan
df_final['D_1_1'] = df_final['D_1_1'].astype(CategoricalDtype(ordered=True))
print(df_final['D_1_1'].value_counts())
print("==========================")
print(df_final['D_1_1'])

# 2주 이상 연속 우울감 여부 [BP5] (명목)
print(df_final['BP5'].value_counts())
print("==========================")
df_final.loc[df_final['BP5'] == 8, 'BP5'] = np.nan
df_final.loc[df_final['BP5'] == 9, 'BP5'] = np.nan
df_final['BP5'] = df_final['BP5'].astype('category')
print(df_final['BP5'].value_counts())
print("==========================")
print(df_final['BP5'])

# 평소 스트레스 인지정도 [BP1] (서열)
print(df_final['BP1'].value_counts())
print("==========================")
df_final.loc[df_final['BP1'] == 8, 'BP1'] = np.nan
df_final.loc[df_final['BP1'] == 9, 'BP1'] = np.nan

df_final['BP1'] = df_final['BP1'].astype(CategoricalDtype(ordered=True))
print(df_final['BP1'].value_counts())
print("==========================")
print(df_final['BP1'])

# 주관적 체형 인지 [BO1] (서열)
print(df_final['BO1'].value_counts())
print("==========================")
df_final.loc[df_final['BO1'] == 8, 'BO1'] = np.nan
df_final.loc[df_final['BO1'] == 9, 'BO1'] = np.nan

df_final['BO1'] = df_final['BO1'].astype(CategoricalDtype(ordered=True))
print(df_final['BO1'].value_counts())
print("==========================")
print(df_final['BO1'])
# 1주일간 근력운동 일수 [BE5_1]] (서열)
print(df_final['BE5_1'].value_counts())
print("==========================")
df_final['BE5_1'] = df_final['BE5_1'].astype(CategoricalDtype(ordered=True))
print(df_final['BE5_1'].value_counts())
print("==========================")
print(df_final['BE5_1'])

df_final['sex'] = df_final['sex'].astype('category')

# 서열 척도 및 명목 척도 적용 후 변수별 데이터 타입 확인
df_final.info()


## 결측치 처리

참고사항 : 
- 결측치가 있는 행 제거, 즉 완전제거법 적용한 데이터셋 이름 : df_all
- 평균값 대체 적용한 데이터셋 이름 : df_mean
- 중앙값 대체 적용한 데이터셋 이름 : df_median
- 최빈값 대체 적용한 데이터셋 이름 : df_mode


In [ ]:
# 결측치 있는 행 제거
df_new = df_final.dropna(axis=0)
df_new = df_new.reset_index()

from sklearn.preprocessing import StandardScaler
transformer = StandardScaler()
standard = transformer.fit_transform(df_new.loc[:,['Total_slp_wd','SitTime']])
df_standard = pd.DataFrame(standard, columns = ['Total_slp_wd_standard','SitTime_standard'])
df_standard = pd.concat([df_new, df_standard], axis = 1)
df_standard

from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
scaled = min_max_scaler.fit_transform(df_new.loc[:,['Total_slp_wd','SitTime']])
df_scaled = pd.DataFrame(scaled, columns = ['Total_slp_wd_scaled','SitTime_scaled'])
df_all = pd.concat([df_standard, df_scaled], axis=1)
df_all = df_all.iloc[:,1:]
df_all
# 평균값 대체
df_csn_mean = df_final.copy()

print(df_csn_mean.isnull().sum())
df_csn_mean.fillna({'D_1_1':int(df_csn_mean['D_1_1'].mode()),
                    'BP1':int(df_csn_mean['BP1'].mode()),
                    'BO1':int(df_csn_mean['BO1'].mode()),
                    'BP5':int(df_csn_mean['BP5'].mode()),
                    'BMI':int(df_csn_mean['BMI'].mode()),
                    'BE5_1':int(df_csn_mean['BE5_1'].mode()),
                    'age':int(df_csn_mean['age'].mean()),
                    'Total_slp_wd':df_csn_mean['Total_slp_wd'].mean(),
                    'SitTime':df_csn_mean['SitTime'].mean()}, inplace=True)
print('========================')
print(df_csn_mean.isnull().sum())

# 평균값 대체 자료에 표준화 및 정규화

# 인덱스 초기화
df_csn_mean.reset_index(inplace=True)
df_csn_mean = df_csn_mean.iloc[:,1:]

## 정규화
from sklearn.preprocessing import StandardScaler
transformer = StandardScaler()
standard = transformer.fit_transform(df_csn_mean.loc[:,['Total_slp_wd','SitTime']])
df_standard = pd.DataFrame(standard, columns = ['Total_slp_wd_standard','SitTime_standard'])

## Min-Max Scaling
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
scaled = min_max_scaler.fit_transform(df_csn_mean.loc[:,['Total_slp_wd','SitTime']])
df_scaled = pd.DataFrame(scaled, columns = ['Total_slp_wd_scaled','SitTime_scaled'])

## 합치기
df_mean = pd.concat([df_csn_mean, df_standard, df_scaled], axis=1)
df_mean.info()

# 중앙값 대체 

df_csn_median = df_final.copy()

print(df_csn_median.isnull().sum())
df_csn_median.fillna({'D_1_1':int(df_csn_median['D_1_1'].mode()),
                    'BP1':int(df_csn_median['BP1'].mode()),
                    'BO1':int(df_csn_median['BO1'].mode()),
                    'BP5':int(df_csn_median['BP5'].mode()),
                    'BMI':int(df_csn_median['BMI'].mode()),
                    'BE5_1':int(df_csn_median['BE5_1'].mode()),
                    'age':int(df_csn_mean['age'].median()),
                    'Total_slp_wd':df_csn_median['Total_slp_wd'].median(),
                    'SitTime':df_csn_median['SitTime'].median()}, inplace=True)
print('========================')
print(df_csn_median.isnull().sum())

## 인덱스 초기화
df_csn_median.reset_index(inplace=True)
df_csn_median = df_csn_median.iloc[:,1:]

## 정규화
from sklearn.preprocessing import StandardScaler
transformer = StandardScaler()
standard = transformer.fit_transform(df_csn_median.loc[:,['Total_slp_wd','SitTime']])
df_standard = pd.DataFrame(standard, columns = ['Total_slp_wd_standard','SitTime_standard'])

## Min-Max Scaling
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
scaled = min_max_scaler.fit_transform(df_csn_median.loc[:,['Total_slp_wd','SitTime']])
df_scaled = pd.DataFrame(scaled, columns = ['Total_slp_wd_scaled','SitTime_scaled'])

## 합치기
df_median = pd.concat([df_csn_median, df_standard, df_scaled], axis=1)
df_median.info()


# 최빈값 대체
df_csn_mode = df_final.copy()

print(df_csn_mode.isnull().sum())
df_csn_mode.fillna({'D_1_1':int(df_csn_mode['D_1_1'].mode()),
                    'BP1':int(df_csn_mode['BP1'].mode()),
                    'BO1':int(df_csn_mode['BO1'].mode()),
                    'BP5':int(df_csn_mode['BP5'].mode()),
                    'BMI':int(df_csn_mode['BMI'].mode()),
                    'BE5_1':int(df_csn_mode['BE5_1'].mode()),
                    'age':int(df_csn_mean['age'].mode()),
                    'Total_slp_wd':float(df_csn_mode['Total_slp_wd'].mode()),
                    'SitTime':float(df_csn_mode['SitTime'].mode())}, inplace=True)
print('========================')
print(df_csn_mode.isnull().sum())

## 인덱스 초기화
df_csn_mode.reset_index(inplace=True)
df_csn_mode = df_csn_mode.iloc[:,1:]

## 정규화
from sklearn.preprocessing import StandardScaler
transformer = StandardScaler()
standard = transformer.fit_transform(df_csn_mode.loc[:,['Total_slp_wd','SitTime']])
df_standard = pd.DataFrame(standard, columns = ['Total_slp_wd_standard','SitTime_standard'])

## Min-Max Scaling
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
scaled = min_max_scaler.fit_transform(df_csn_mode.loc[:,['Total_slp_wd','SitTime']])
df_scaled = pd.DataFrame(scaled, columns = ['Total_slp_wd_scaled','SitTime_scaled'])

## 합치기
df_mode = pd.concat([df_csn_mode, df_standard, df_scaled], axis=1)
df_mode.info()

# csv 파일로 저장하기
df_all.to_csv(‘df_all.csv’)
df_mean.to_csv(‘df_mean.csv’)
df_median.to_csv(‘df_median.csv’)
df_mode.to_csv(‘df_mode.csv’)

# 결측치 처리한 데이터별 Instance 및 변수 개수 확인
print(df_all.shape)
print(df_mean.shape)
print(df_median.shape)
print(df_mode.shape)

#남자 데이터 만들기
df_all_b = df_all.loc[df_all["sex"]==1,:]
df_all_b = df_all_b.drop('sex', axis = 1)

df_mean_b = df_mean.loc[df_mean["sex"]==1,:]
df_mean_b = df_mean_b.drop('sex', axis = 1)

df_median_b = df_median.loc[df_median["sex"]==1,:]
df_median_b = df_median_b.drop('sex', axis = 1)

df_mode_b = df_mode.loc[df_mode["sex"]==1,:]
df_mode_b = df_mode_b.drop('sex', axis = 1)

df_all_b.shape, df_mean_b.shape, df_median_b.shape, df_mode_b.shape

#여자 데이터 만들기
df_all_g = df_all.loc[df_all["sex"]==2,:]
df_all_g = df_all_g.drop('sex', axis = 1)

df_mean_g = df_mean.loc[df_mean["sex"]==2,:]
df_mean_g = df_mean_g.drop('sex', axis = 1)

df_median_g = df_median.loc[df_median["sex"]==2,:]
df_median_g = df_median_g.drop('sex', axis = 1)

df_mode_g = df_mode.loc[df_mode["sex"]==2,:]
df_mode_g = df_mode_g.drop('sex', axis = 1)

df_all_g.shape, df_mean_g.shape, df_median_g.shape, df_mode_g.shape

# 분석 대상 데이터 불러오기

In [ ]:
# 결측치 행 제거 파일
df_all = pd.read_csv('/content/drive/My Drive/고급데이터마이닝/df_all.csv')
df_all = df_all.iloc[:,1:]
df_all

for i in df_all.columns:
  if i in ['Total_slp_wd','SitTime','Total_slp_wd_standard','SitTime_standard','Total_slp_wd_scaled','SitTime_scaled']:
    pass
  else:
    df_all[i] = df_all[i].astype('category')

# 남녀 데이터 생성
df_all_b = df_all.loc[df_all["sex"]==1,:]
df_all_g = df_all.loc[df_all["sex"]==2,:]
df_all_b = df_all_b.drop('sex', axis = 1)
df_all_g = df_all_g.drop('sex', axis = 1)

# 평균값 대체 파일
df_mean = pd.read_csv('/content/drive/My Drive/고급데이터마이닝/평균, 중앙값, 최빈값 대체 데이터/df_mean.csv')
df_mean = df_mean.iloc[:,1:]
df_mean

for i in df_mean.columns:
  if i in ['Total_slp_wd','SitTime','Total_slp_wd_standard','SitTime_standard','Total_slp_wd_scaled','SitTime_scaled']:
    pass
  else:
    df_mean[i] = df_mean[i].astype('category')

# 남녀 데이터 생성 
df_mean_b = df_mean.loc[df_mean["sex"]==1,:]
df_mean_g = df_mean.loc[df_mean["sex"]==2,:]
df_mean_b = df_mean_b.drop('sex', axis = 1)
df_mean_g = df_mean_g.drop('sex', axis = 1)




# 중앙값 대체 파일
df_median = pd.read_csv('/content/drive/My Drive/고급데이터마이닝/평균, 중앙값, 최빈값 대체 데이터/df_median.csv')
df_median = df_median.iloc[:,1:]
df_median

for i in df_median.columns:
  if i in ['Total_slp_wd','SitTime','Total_slp_wd_standard','SitTime_standard','Total_slp_wd_scaled','SitTime_scaled']:
    pass
  else:
    df_median[i] = df_median[i].astype('category')

# 남녀 데이터 생성 
df_median_b = df_median.loc[df_median["sex"]==1,:]
df_median_g = df_median.loc[df_median["sex"]==2,:]
df_median_b = df_median_b.drop('sex', axis = 1)
df_median_g = df_median_g.drop('sex', axis = 1)


# 최빈값 대체 파일
df_mode = pd.read_csv('/content/drive/My Drive/고급데이터마이닝/평균, 중앙값, 최빈값 대체 데이터/df_mode.csv')
df_mode = df_mode.iloc[:,1:]
df_mode

for i in df_mode.columns:
    if i in ['Total_slp_wd','SitTime','Total_slp_wd_standard','SitTime_standard','Total_slp_wd_scaled','SitTime_scaled']:
        pass
      else:
        df_mode[i] = df_mode[i].astype('category')

# 남녀 데이터 생성 
df_mode_b = df_mode.loc[df_mode["sex"]==1,:]
df_mode_g = df_mode.loc[df_mode["sex"]==2,:]
df_mode_b = df_mode_b.drop('sex', axis = 1)
df_mode_g = df_mode_g.drop('sex', axis = 1)

# Modeling & Evaluation

## 나이브 베이즈 분류기

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV

def model_evaluation(label, predict):
    cf_matrix = confusion_matrix(label, predict)
    Accuracy = (cf_matrix[0][0] + cf_matrix[1][1]) / sum(sum(cf_matrix))
    Precision = cf_matrix[1][1] / (cf_matrix[1][1] + cf_matrix[0][1])
    Recall = cf_matrix[1][1] / (cf_matrix[1][1] + cf_matrix[1][0])
    Specificity = cf_matrix[0][0] / (cf_matrix[0][0] + cf_matrix[0][1])
    F1_Score = (2 * Recall * Precision) / (Recall + Precision)
    
    print("Accuracy: ", Accuracy)
    print("Precision: ", Precision)
    print("Recall: ", Recall)
    print("Specificity: ", Specificity)
    print("F1-Score: ", F1_Score)

count = 0

for df in [df_all_g, df_mean_g, df_median_g, df_mode_g]:
    for column in [['Total_slp_wd_scaled','SitTime_scaled','Total_slp_wd_standard','SitTime_standard','BMI'],
                                                    ['Total_slp_wd_scaled','SitTime_scaled','Total_slp_wd','SitTime','BMI'],
                                                    ['Total_slp_wd_standard','SitTime_standard','Total_slp_wd','SitTime','BMI']]:
        
        count = count+1
        
        if count == 1 or count  == 2 or count == 3:
            print('제목: df_all_g')
        elif count == 4 or count  == 5 or count == 6:
            print('제목: df_mean_g')
        elif count == 7 or count  == 8 or count == 9:
            print('제목: df_median_g')
        elif count == 10 or count  == 11 or count == 12:
            print('제목: df_mode_g')

        
        if count == 1 or count  == 4 or count == 7 or count == 10 :
            print('데이터: 원본')
        elif count == 2 or count  == 5 or count == 8 or count == 11:
            print('제목: Standard')
        elif count == 3 or count  == 6 or count == 9 or count == 12:
            print('제목: Scaled')

        
        X = df.drop(column,axis=1) 
        y = df['BMI']

        # test_size : 테스트 데이터셋의 비율(float)이나 갯수(int) 
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y, random_state=3)

        # 파라메터 후보
        param_grid = {
            'var_smoothing': [1e-8, 1e-9, 1e-10]
        }

        # 그리드 서치 진행
        grid_search = GridSearchCV(GaussianNB(), param_grid, cv=10)        

        grid_search.fit(X_train, y_train)
        grid_search.score(X_test, y_test)
        print("best parameters : {}".format(grid_search.best_params_))
        
        predicted = grid_search.predict(X_test)
        print(confusion_matrix(y_test, predicted))
        print(model_evaluation(y_test, predicted))

        #print('최고 예측 정확도: {:.4f}'.format(grid_cv.best_score_))
        print('='*40)
        print()
        print()

## 랜덤 포레스트

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold, GridSearchCV

def model_evaluation(label, predict):
    cf_matrix = confusion_matrix(label, predict)
    Accuracy = (cf_matrix[0][0] + cf_matrix[1][1]) / sum(sum(cf_matrix))
    Precision = cf_matrix[1][1] / (cf_matrix[1][1] + cf_matrix[0][1])
    Recall = cf_matrix[1][1] / (cf_matrix[1][1] + cf_matrix[1][0])
    Specificity = cf_matrix[0][0] / (cf_matrix[0][0] + cf_matrix[0][1])
    F1_Score = (2 * Recall * Precision) / (Recall + Precision)
    
    print("Accuracy: ", round(Accuracy,3))
    print("Precision: ", round(Precision,3))
    print("Recall: ", round(Recall,3))
    print("Specificity: ", round(Specificity,3))
    print("F1-Score: ", round(F1_Score,3))

count = 0

for df in [df_all_b, df_mean_b, df_median_b, df_mode_b]:
    for column in [['Total_slp_wd_scaled','SitTime_scaled','Total_slp_wd_standard','SitTime_standard','BMI'],
                                                    ['Total_slp_wd_scaled','SitTime_scaled','Total_slp_wd','SitTime','BMI'],
                                                    ['Total_slp_wd_standard','SitTime_standard','Total_slp_wd','SitTime','BMI']]:
        
        count = count+1
        
        if count == 1 or count  == 2 or count == 3:
            print('제목: df_all_b')
        elif count == 4 or count  == 5 or count == 6:
            print('제목: df_mean_b')
        elif count == 7 or count  == 8 or count == 9:
            print('제목: df_median_b')
        elif count == 10 or count  == 11 or count == 12:
            print('제목: df_mode_b')
        
        if count == 1 or count  == 4 or count == 7 or count == 10 :
            print('데이터: 원본')
        elif count == 2 or count  == 5 or count == 8 or count == 11:
            print('제목: Standard')
        elif count == 3 or count  == 6 or count == 9 or count == 12:
            print('제목: Scaled')
        
        X = df.drop(column,axis=1) 
        y = df['BMI']

        # test_size : 테스트 데이터셋의 비율(float)이나 갯수(int) 
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y, random_state=3)

        #최적 파라미터 값 찾기
        params = { 'n_estimators' : [10, 100],
                   'max_depth' : [6, 8, 10, 12],
                   'min_samples_leaf' : [8, 12, 18],
                   'min_samples_split' : [8, 16, 20]
                    }
        rf_clf = RandomForestClassifier(random_state = 0, n_jobs = -1)
        grid_cv = GridSearchCV(rf_clf, param_grid = params, cv = 5, n_jobs = -1)
        grid_cv.fit(X_train, y_train)

        predicted = grid_cv.predict(X_test)
        print(confusion_matrix(y_test, predicted))
        print(model_evaluation(y_test, predicted))

        print('하이퍼 파라미터: ', grid_cv.best_params_)
        #print('최고 예측 정확도: {:.4f}'.format(grid_cv.best_score_))
        print('='*40)
        print()

## 로지스틱 회귀

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, GridSearchCV


count = 0

for df in [df_all, df_mean, df_median, df_mode]:
    for column in [['Total_slp_wd_scaled','SitTime_scaled','Total_slp_wd_standard','SitTime_standard','BMI'],
                                                    ['Total_slp_wd_scaled','SitTime_scaled','Total_slp_wd','SitTime','BMI'],
                                                    ['Total_slp_wd_standard','SitTime_standard','Total_slp_wd','SitTime','BMI']]:
        
        count = count+1
        
        if count == 1 or count  == 2 or count == 3:
            print('제목: df_all')
        elif count == 4 or count  == 5 or count == 6:
            print('제목: df_mean')
        elif count == 7 or count  == 8 or count == 9:
            print('제목: df_median')
        elif count == 10 or count  == 11 or count == 12:
            print('제목: df_mode')

        
        if count == 1 or count  == 4 or count == 7 or count == 10 :
            print('데이터: 원본')
        elif count == 2 or count  == 5 or count == 8 or count == 11:
            print('제목: Standard')
        elif count == 3 or count  == 6 or count == 9 or count == 12:
            print('제목: Scaled')
  
        
        X = df.drop(column,axis=1) 
        y = df['BMI']

        # test_size : 테스트 데이터셋의 비율(float)이나 갯수(int) 
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y, random_state=3)

        # 파라메터 후보
        param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100],
              'penalty': ['l1', 'l2']}

        # 그리드 서치 진행
        grid_search = GridSearchCV(LogisticRegression(solver='liblinear', random_state = 42), param_grid, cv= 5 )        

        grid_search.fit(X_train, y_train)
        grid_search.score(X_test, y_test)
        print("best parameters : {}".format(grid_search.best_params_))
        
        predicted = grid_search.predict(X_test)
        print(confusion_matrix(y_test, predicted))
        print(model_evaluation(y_test, predicted))

        #print('최고 예측 정확도: {:.4f}'.format(grid_cv.best_score_))
        print('='*40)
        print()
        print()

## K-NN

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

def model_evaluation(label, predict):
    cf_matrix = confusion_matrix(label, predict)
    Accuracy = (cf_matrix[0][0] + cf_matrix[1][1]) / sum(sum(cf_matrix))
    Precision = cf_matrix[1][1] / (cf_matrix[1][1] + cf_matrix[0][1])
    Recall = cf_matrix[1][1] / (cf_matrix[1][1] + cf_matrix[1][0])
    Specificity = cf_matrix[0][0] / (cf_matrix[0][0] + cf_matrix[0][1])
    F1_Score = (2 * Recall * Precision) / (Recall + Precision)
    
    print("Accuracy: ", round(Accuracy,3))
    print("Precision: ", round(Precision,3))
    print("Recall: ", round(Recall,3))
    print("Specificity: ", round(Specificity,3))
    print("F1-Score: ", round(F1_Score,3))

count = 0

for df in [df_all_g, df_mean_g, df_median_g, df_mode_g]:
    for column in [['Total_slp_wd_scaled','SitTime_scaled','Total_slp_wd_standard','SitTime_standard','BMI'],
                                                    ['Total_slp_wd_scaled','SitTime_scaled','Total_slp_wd','SitTime','BMI'],
                                                    ['Total_slp_wd_standard','SitTime_standard','Total_slp_wd','SitTime','BMI']]:
        
        count = count+1
        
        if count == 1 or count  == 2 or count == 3:
            print('제목: df_all_g')
        elif count == 4 or count  == 5 or count == 6:
            print('제목: df_mean_g')
        elif count == 7 or count  == 8 or count == 9:
            print('제목: df_median_g')
        elif count == 10 or count  == 11 or count == 12:
            print('제목: df_mode_g')
        
        if count == 1 or count  == 4 or count == 7 or count == 10 :
            print('데이터: 원본')
        elif count == 2 or count  == 5 or count == 8 or count == 11:
            print('제목: Standard')
        elif count == 3 or count  == 6 or count == 9 or count == 12:
            print('제목: Scaled')
        
        X = df.drop(column,axis=1) 
        y = df['BMI']

        # test_size : 테스트 데이터셋의 비율(float)이나 갯수(int) 
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y, random_state=3)



        #최적 파라미터 값 찾기
        knn_classifier = KNeighborsClassifier()        
        knn_parameters = [{
            'n_neighbors': [1, 3, 5, 7, 9, 11, 13, 15],
            'leaf_size': [5, 10, 15, 20, 25, 30, 35, 40],
            'weights': ['uniform', 'distance']}]
        # Stratified k-fold (default for classifier)
        # n = 5 folds is default
        grid_cv = GridSearchCV(estimator = knn_classifier, param_grid = knn_parameters, cv=5, scoring='recall')
        grid_cv.fit(X_train, y_train)

        predicted = grid_cv.predict(X_test)
        print(confusion_matrix(y_test, predicted))
        print(model_evaluation(y_test, predicted))

        print('최적 하이퍼 파라미터: ', grid_cv.best_params_)
        #print('최고 예측 정확도: {:.4f}'.format(grid_cv.best_score_))
        print('='*40)
        print()

## SGD Classifier

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDClassifier

count = 0

for df in [df_median, df_mode]:
    for column in [['Total_slp_wd_scaled','SitTime_scaled','Total_slp_wd_standard','SitTime_standard','BMI'],
                                                    ['Total_slp_wd_scaled','SitTime_scaled','Total_slp_wd','SitTime','BMI'],
                                                    ['Total_slp_wd_standard','SitTime_standard','Total_slp_wd','SitTime','BMI']]:
        
        count = count+1
        
        if count >= 1 & count <=3:
            print('제목: df_all_b')
        elif count>=4 & count <=6:
            print('제목: df_mean_b')
        elif count>=7 & count <=9:
            print('제목: df_median_b')
        elif count >= 10 & count <= 12:
            print('제목: df_mode_b')
        
        if count == 1 or count  == 4 or count == 7 or count == 10 :
            print('데이터: 원본')
        elif count == 2 or count  == 5 or count == 8 or count == 11:
            print('제목: Standard')
        elif count == 3 or count  == 6 or count == 9 or count == 12:
            print('제목: Scaled')

            
            
        
        
        X = df.drop(column,axis=1) 
        y = df['BMI']

        # test_size : 테스트 데이터셋의 비율(float)이나 갯수(int) 
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y, random_state=3)

        #최적 파라미터 값 찾기
        params = {"loss" : ["hinge", "log", "squared_hinge", "modified_huber"],
                  "alpha" : [0.0001, 0.001, 0.01, 0.1],
                  "penalty" : ["l2", "l1", "none"],
 }
        model = SGDClassifier(max_iter=1000)
        clf = GridSearchCV(model, param_grid=params, cv=10)
        clf.fit(X_train, y_train)

        predicted = clf.predict(X_test)
        print(confusion_matrix(y_test, predicted))
        print(model_evaluation(y_test, predicted))

        print('하이퍼 파라미터: ', clf.best_params_)
        #print('최고 예측 정확도: {:.4f}'.format(grid_cv.best_score_))
        print('='*40)
        print()
        print()

## Support Vector Machine

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

count = 0

for df in [df_median, df_mode]:
    for column in [                               ['Total_slp_wd_scaled','SitTime_scaled','Total_slp_wd','SitTime','BMI'],
                                                    ['Total_slp_wd_standard','SitTime_standard','Total_slp_wd','SitTime','BMI']]:
        
        count = count+1
        
        if count >= 1 & count <=3:
            print('제목: df_all_b')
        elif count>=4 & count <=6:
            print('제목: df_mean_b')
        elif count>=7 & count <=9:
            print('제목: df_median_b')
        elif count >= 10 & count <= 12:
            print('제목: df_mode_b')
        
        if count == 1 or count  == 4 or count == 7 or count == 10 :
            print('데이터: 원본')
        elif count == 2 or count  == 5 or count == 8 or count == 11:
            print('제목: Standard')
        elif count == 3 or count  == 6 or count == 9 or count == 12:
            print('제목: Scaled')

            
            
        
        
        X = df.drop(column,axis=1) 
        y = df['BMI']

        # test_size : 테스트 데이터셋의 비율(float)이나 갯수(int) 
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y, random_state=3)


        #최적 파라미터 값 찾기
        param_grid = {'C':[0.001, 0.01, 0.1], 'gamma':[0.01, 0.1,1]}

        grid_search = GridSearchCV(SVC(),param_grid, cv=5, return_train_score = True)
        grid_search.fit(X_train, y_train)

        print("best parameters : {}".format(grid_search.best_params_))
        
        predicted = grid_search.predict(X_test)
        print(confusion_matrix(y_test, predicted))
        print(model_evaluation(y_test, predicted))

        #print('최고 예측 정확도: {:.4f}'.format(grid_cv.best_score_))
        print('='*40)
        print()
        print()